In [1]:
import torch, platform, time, os, random
print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
torch.backends.cudnn.benchmark = True  # Enable GPU acceleration


CUDA available: True
Tesla T4


In [2]:
!pip install -q timm torch torchvision torchaudio


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import timm, math
random.seed(42); torch.manual_seed(42)


In [4]:
IMG_SIZE = 128  # Smaller than 224
BATCH_SIZE = 64

transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_full = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform)
test_full  = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform)

# Quick subset: 10k training samples, 2k testing samples
train_idx = list(range(0, min(10000, len(train_full))))
test_idx  = list(range(0, min(2000, len(test_full))))

train_ds = Subset(train_full, train_idx)
test_ds  = Subset(test_full, test_idx)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)


100%|██████████| 170M/170M [00:04<00:00, 36.1MB/s]


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

models = {
    "ResNet18": timm.create_model('resnet18', pretrained=False, num_classes=10).to(device),
    "DenseNet121": timm.create_model('densenet121', pretrained=False, num_classes=10).to(device),
    "EfficientNetB0": timm.create_model('efficientnet_b0', pretrained=False, num_classes=10).to(device)
}


In [6]:
from torch.cuda.amp import autocast, GradScaler

def train_model_fast(model, train_loader, test_loader, epochs=1, max_steps_per_epoch=150):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scaler = GradScaler(enabled=torch.cuda.is_available())

    for epoch in range(epochs):
        model.train()
        running_loss, steps = 0.0, 0
        t0 = time.time()
        for images, labels in train_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=torch.cuda.is_available()):
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            steps += 1
            if steps >= max_steps_per_epoch:  # Reduce time per epoch
                break
        t1 = time.time()
        print(f"Epoch {epoch+1} | Loss: {running_loss/steps:.4f} | Time: {t1-t0:.1f}s")

    model.eval()
    correct, total = 0, 0
    with torch.no_grad(), autocast(enabled=torch.cuda.is_available()):
        for images, labels in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
    acc = 100.0 * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    return acc


In [7]:
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    acc = train_model_fast(model, train_loader, test_loader, epochs=1, max_steps_per_epoch=150)
    results[name] = acc

print("\nFinal Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}%")



Training ResNet18...


/tmp/ipython-input-3267225029.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=torch.cuda.is_available())
/tmp/ipython-input-3267225029.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=torch.cuda.is_available()):


Epoch 1 | Loss: 1.8124 | Time: 20.7s


/tmp/ipython-input-3267225029.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast(enabled=torch.cuda.is_available()):


Test Accuracy: 40.60%

Training DenseNet121...
Epoch 1 | Loss: 1.6975 | Time: 33.3s
Test Accuracy: 44.55%

Training EfficientNetB0...
Epoch 1 | Loss: 2.6579 | Time: 49.8s
Test Accuracy: 27.35%

Final Results:
ResNet18: 40.60%
DenseNet121: 44.55%
EfficientNetB0: 27.35%
